In [1]:
!pip install tensorflow keras opencv-python numpy matplotlib scikit-learn


In [2]:
pip install selenium beautifulsoup4 opencv-python requests


Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# Create dataset folder
dataset_path = "E:\\twin_face\\twin_images"
os.makedirs(dataset_path, exist_ok=True)

# Set up Selenium WebDriver
driver = webdriver.Chrome()  # Ensure you have ChromeDriver installed
driver.get("https://www.google.com/imghp")

# Search for twin images
search_box = driver.find_element(By.NAME, "q")
search_box.send_keys("identical twins face images")
search_box.send_keys(Keys.RETURN)

# Wait for images to load
time.sleep(2)

# Get image elements
image_elements = driver.find_elements(By.CSS_SELECTOR, "img")

# Download first 50 images
count = 0
for img in image_elements[:200]:
    img_url = img.get_attribute("src")
    if img_url and "http" in img_url:
        response = requests.get(img_url)
        with open(f"{dataset_path}/twin_{count}.jpg", "wb") as file:
            file.write(response.content)
        count += 1

print(f"Downloaded {count} twin images!")
driver.quit()



Downloaded 2 twin images!


In [5]:
import cv2
import os

# Load the image
image_path = "E:\\twin_face\\twin_images\\twin_0.jpg"  # Change this to the correct file path
image = cv2.imread(image_path)

# Load OpenCV's pre-trained face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Convert image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Detect faces
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))

# Create output folder
output_folder = "E:\\twin_face\\twin_images\\separated_faces"
os.makedirs(output_folder, exist_ok=True)

# Extract and save faces
for i, (x, y, w, h) in enumerate(faces):
    face = image[y:y+h, x:x+w]  # Crop face
    face_path = os.path.join(output_folder, f"twin_{i+1}.jpg")
    cv2.imwrite(face_path, face)
    print(f"Saved: {face_path}")

print("Twins separated successfully!")


Twins separated successfully!


In [6]:
import cv2
import numpy as np
import os

def load_images_from_folder(folder, img_size=(224, 224)):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, img_size)
            images.append(img)
            labels.append(filename.split('_')[1])  # Assuming file names contain twin IDs
    return np.array(images), np.array(labels)

images, labels = load_images_from_folder("E:\\twins_faces\\twin_images\\separated_faces\\pic_11")


In [7]:
import os
import cv2
import numpy as np
import re

def load_images_from_folder(folder, img_size=(224, 224)):
    images = []
    labels = []
    
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)

        if img is not None:
            img = cv2.resize(img, img_size)
            images.append(img)

            # Extract number after underscore using regex
            match = re.search(r'_(\d+)', filename)
            if match:
                labels.append(match.group(1))
            else:
                print(f"Skipping: {filename} (No label found)")
                continue
    
    return np.array(images), np.array(labels)

In [8]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model

# Load MobileNetV2 model as a feature extractor
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
feature_extractor = Model(inputs=base_model.input, outputs=base_model.output)

# Extract features from images
features = feature_extractor.predict(images)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


In [9]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

# Define a distance function (L2 norm)
def euclidean_distance(vectors):
    x, y = vectors
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

input_shape = (7, 7, 1280)  # Shape of MobileNetV2 output

# Twin input layers
input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

# Flatten features
flat_a = Flatten()(input_a)
flat_b = Flatten()(input_b)

# Compute the distance
distance = Lambda(euclidean_distance)([flat_a, flat_b])

# Fully connected layer
fc = Dense(1, activation="sigmoid")(distance)

# Build the model
siamese_model = Model(inputs=[input_a, input_b], outputs=fc)
siamese_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
siamese_model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)    │ (None, 7, 7, 1280)        │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_2 (InputLayer)    │ (None, 7, 7, 1280)        │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten (Flatten)             │ (None, 62720)             │               0 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten_1 (Flatten)           │ (None, 62720)             │               0 │ input_layer_2[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lambda (Lambda)               │ (None, 1)                 │               0 │ flatten[0][0],             │
│                               │                           │                 │ flatten_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 1)                 │               2 │ lambda[0][0]               │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2 (8.00 B)

 Trainable params: 2 (8.00 B)

 Non-trainable params: 0 (0.00 B)

In [12]:
# Prepare positive (same twin) and negative (different twins) pairs
def create_pairs(images, labels):
    pairs = []
    targets = []
    label_dict = {label: [] for label in set(labels)}
    for i, label in enumerate(labels):
        label_dict[label].append(images[i])

    # Create positive and negative pairs
    for label in label_dict:
        for i in range(len(label_dict[label]) - 1):
            pairs.append([label_dict[label][i], label_dict[label][i + 1]])
            targets.append(1)  # Same twin = 1

        for neg_label in label_dict:
            if neg_label != label:
                pairs.append([label_dict[label][0], label_dict[neg_label][0]])
                targets.append(0)  # Different twins = 0

    return np.array(pairs), np.array(targets)

pairs, targets = create_pairs(features, labels)

# Train the Siamese network
siamese_model.fit([pairs[:, 0], pairs[:, 1]], targets, batch_size=16, epochs=10)


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.0000e+00 - loss: 174.7863
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - accuracy: 0.0000e+00 - loss: 174.6291
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.0000e+00 - loss: 174.4719
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 0.0000e+00 - loss: 174.3147
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.0000e+00 - loss: 174.1575
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 0.0000e+00 - loss: 174.0002
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - accuracy: 0.0000e+00 - loss: 173.8430
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.0000e+00 - loss: 173.6858
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - accuracy: 0.0000e+00 - loss: 173.5286
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - accuracy: 0.0000e+00 - loss: 173.3714


In [11]:
import cv2
import numpy as np

# Define the required input shape for the feature extractor
INPUT_SHAPE = (224, 224)  # Adjust according to your model's expected size

def preprocess_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, INPUT_SHAPE)  # Resize image to match model input
    img = img.astype("float32") / 255.0  # Normalize pixel values
    return img

def predict_twin_similarity(img1_path, img2_path):
    img1 = preprocess_image(img1_path)
    img2 = preprocess_image(img2_path)
    
    img1_features = feature_extractor.predict(np.expand_dims(img1, axis=0))
    img2_features = feature_extractor.predict(np.expand_dims(img2, axis=0))
    
    similarity = siamese_model.predict([img1_features, img2_features])[0][0]
    return similarity

# Load and compare twins' images
img1_path = "E:\\twins_faces\\twin_images\\separated_faces\\pic_11\\twin_1.jpg"
img2_path = "E:\\twins_faces\\twin_images\\separated_faces\\pic_11\\twin_2.jpg"

similarity_score = predict_twin_similarity(img1_path, img2_path)
print(f"Similarity Score: {similarity_score}")

if similarity_score > 0.5:
    print("Same Twin")
else:
    print("Different Twins")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
Similarity Score: 1.0
Same Twin


In [27]:
# After training your model
feature_extractor.save("E:/twin_face/feature_extractor.h5")


In [28]:
siamese_model.save("E:/twin_face/siamese_model.h5")


In [68]:
!pip install flask flask-ngrok opencv-python numpy


In [23]:
import numpy as np
import cv2
import tensorflow as tf
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)  # Use NGROK for public access if needed

# Load your model (assuming it is pre-trained and available)
feature_extractor = tf.keras.models.load_model("feature_extractor.h5")
siamese_model = tf.keras.models.load_model("siamese_model.h5")

def predict_twin_similarity(img1, img2):
    img1 = feature_extractor.predict(np.expand_dims(img1, axis=0))
    img2 = feature_extractor.predict(np.expand_dims(img2, axis=0))
    similarity = siamese_model.predict([img1, img2])[0][0]
    return similarity

@app.route('/predict', methods=['POST'])
def predict():
    file1 = request.files.get('img1_path = "E:\\twin_face\\twin_images\\separated_faces\\pic_11\\twin_1.jpg"')
    file2 = request.files.get('img1_path = "E:\\twin_face\\twin_images\\separated_faces\\pic_11\\twin_2.jpg"')

    if not file1 or not file2:
        return jsonify({"error": "Both images are required"}), 400

    img1 = cv2.imdecode(np.frombuffer(file1.read(), np.uint8), cv2.IMREAD_COLOR)
    img2 = cv2.imdecode(np.frombuffer(file2.read(), np.uint8), cv2.IMREAD_COLOR)

    if img1 is None or img2 is None:
        return jsonify({"error": "Invalid image format"}), 400

    similarity_score = predict_twin_similarity(img1, img2)
    
    result = "Same Twin" if similarity_score > 0.5 else "Different Twins"
    return jsonify({"similarity_score": float(similarity_score), "result": result})

if __name__ == '__main__':
    app.run()


TypeError: Could not locate function 'euclidean_distance'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'module': 'builtins', 'class_name': 'function', 'config': 'euclidean_distance', 'registered_name': 'function'}

In [43]:
import os
print(os.listdir())  # List files in the current directory


['.ipynb_checkpoints', 'doc', 'twin_images', 'twin_recognition.ipynb']


In [44]:
import tensorflow as tf

# Provide the full path to the models
feature_extractor = tf.keras.models.load_model("E:/twin_face/feature_extractor.h5")
siamese_model = tf.keras.models.load_model("E:/twin_face/siamese_model.h5")


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'E:/twin_face/feature_extractor.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [45]:
import os

file_path = "E:/twin_face/feature_extractor.h5"
print("File exists:", os.path.exists(file_path))


File exists: False


In [46]:
print(os.listdir("E:\\twin_face\\"))


['.ipynb_checkpoints', 'doc', 'twin_images', 'twin_recognition.ipynb']


In [47]:
feature_extractor = tf.keras.models.load_model("E:/twin_face/feature_extractor.keras")


ValueError: File not found: filepath=E:/twin_face/feature_extractor.keras. Please ensure the file is an accessible `.keras` zip file.

In [48]:
feature_extractor = tf.keras.models.load_model(r"E:\\twin_face\\feature_extractor.h5")


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'E:\\twin_face\\feature_extractor.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)